In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import pandas as pd, numpy as np 
from travis_attack.config import Config
from travis_attack.insights import get_training_dfs, _prepare_df_concat
from travis_attack.utils import display_all
from IPython.core.debugger import set_trace
import logging 
logger = logging.getLogger("run")

In [ ]:
cfg = Config()
#run_id = 'cdy5d5yu'
run_name = "cerulean-smoke-317"
path_run = f"{cfg.path_checkpoints}{run_name}/"

## Looking at some examples 

Below we just look at some examples to get a feel for what is going on. 

In [ ]:
def get_interesting_idx(df, n):
    def get_idx_with_top_column_values(cname, n=5, ascending=False):
        return df[['idx',cname]].\
            drop_duplicates().\
            sort_values(cname, ascending=ascending)\
            ['idx'][0:n].values.tolist()
    
    def sample_idx_with_label_flips(n=5): 
        df1 = df[['idx','label_flip']].query("label_flip!=0")
        if len(df1) == 0 : print("No label flips detected"); return None
        else: return df1.drop_duplicates()['idx'].sample(n).values.tolist()
    
    idx_d = dict(
        random = df.idx.drop_duplicates().sample(n).tolist(),
        label_flips = sample_idx_with_label_flips(n=n),
        idx_n_unique_pp  = get_idx_with_top_column_values('idx_n_unique_pp',n=n,ascending=False),
       # idx_n_pp_changes = get_idx_with_top_column_values('idx_n_pp_changes',n=n,ascending=False),
        low_sts = get_idx_with_top_column_values('sts_score',n=n,ascending=True)
    )
    return idx_d

def print_stats(df, idx_d, key, i):
    print("\n###############\n")
    print(key, i, "\n")
    if idx_d[key] is None: return
    idx = idx_d[key][i]
    # Setup 
    df1 = df.query('idx==@idx')
    orig = pd.unique(df1['orig_l'])[0]
    print("Original:", orig)
    print("Original label", pd.unique(df1['orig_label'])[0] )
    pp_all = list(df1['pp_l'])
    #print("All paraphrases", pp_all)
    pp_unique = list(pd.unique(df1['pp_l']))
    n_pp_unique = len(pp_unique)

    # showing a "timeline" of how the paraphrases change over the epochs
    g_fields = ["pp_l","pp_truelabel_probs","vm_score","sts_score","reward","label_flip"]
    #g_fields = ["pp_l","vm_score"]
    g = df1.groupby(g_fields).agg({'epoch' : lambda x: list(x)})
    g = g.sort_values(by='epoch', key = lambda col: col.map(lambda x: np.min(x)))
    print("Unique paraphrases:", n_pp_unique)
    print("How the paraphrases change:")
    display_all(g)

    # Showing a dataframe of the few best paraphrases
    best_pps = df1.sort_values('reward', ascending=False).iloc[0]
    print("Best Paraphrase")
    display_all(best_pps.to_frame().T)
        
def print_interesting_text_stats(df, n): 
    idx_d = get_interesting_idx(df, n)
    for key in idx_d.keys():
        for i in range(n): 
            print_stats(df, idx_d, key,i)

In [ ]:
split = 'training_step'
df_d = get_training_dfs(path_run, postprocessed=True)
idx_d = get_interesting_idx(df_d[split], n=5)
print_interesting_text_stats(df_d[split], n=5)


###############

random 0 

Original: the direction occasionally rises to the level of marginal competence , but for most of the film it is hard to tell who is chasing who or why .
Original label 0
Unique paraphrases: 2
How the paraphrases change:


,,,,,,epoch
pp_l,pp_truelabel_probs,vm_score,sts_score,reward,label_flip,
"The direction occasionally rises to the level of marginal competence, but for most of the film it is hard to tell who is chasing who or why.",0.89919,0.00000,0.99977,0.20000,0,"[1, 2, 4, 5]"
"The direction occasionally rises to the level of marginal competence, but for most of the film is hard to tell who is chasing who or why.",0.91416,-0.01497,0.99898,0.08024,0,[3]


Best Paraphrase


,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
19990,4572,1,"the direction occasionally rises to the level of marginal competence , but for most of the film it is hard to tell who is chasing who or why .","The direction occasionally rises to the level of marginal competence, but for most of the film it is hard to tell who is chasing who or why.",0.899185,0.89919,0.899185,0,0,0,0,0.99977,0.2,-4.43223,0.886446,154,154,2,61.5281,0.961376,0.125,32,16,60,16,0.867072,0.156296,0.0780809,0.00801183,0.0669606,0.077801,0.0183736,0.0342745,5.59464e-05,1.52098,0.0999728,1.90362e-05,1.45286e-06,2,2,1,27,1,2,0,112,27,1,2,0,112,0,0,0,0,0,[0],[the],[1],[The],"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]","[direction occasionally rises to the level of marginal competence, but for most of the film it is hard to tell who is chasing who or why.]",1,1,1,1,False,True,False,1



###############

random 1 

Original: a chilling tale of one of the great crimes of 20th century france : the murder of two rich women by their servants in 1933 .
Original label 1
Unique paraphrases: 4
How the paraphrases change:


...

,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
1672,371,3,"romanek keeps the film constantly taut . . . reflecting the character's instability with a metaphorical visual style and an unnerving , heartbeat-like score .","romanek keeps the film constantly taut, reflecting the character's instability with a metaphorical visual style and an unnerving heartbeat-like score.",0.943847,0.93828,0.938284,1,1,0,0.00556,0.98439,0.24451,-1.00127,0.244816,111,1043,3,3.33975,0.0521835,0,40,16,36,16,0.576689,0.138407,0.081044,0.00787122,0.0685986,0.057171,0.0106602,0.0298826,6.33523e-05,0.907469,0.121391,0.208442,0.104465,3,2,2,21,1,7,0,127,21,1,4,0,127,0,0,3,0,0,"[7, 8, 9, 23]","[..., ,]",[6],"[,]","[0, 1, 2, 3, 4, 5, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28]","[romanek keeps the film constantly taut, reflecting the character 's instability with a metaphorical visual style and an unnerving, heartbeat- like score.]",1,2,1,4,False,True,True,4



###############

low_sts 4 

Original: the filmmakers skillfully evoke the sense of menace that nature holds for many urban dwellers .
Original label 1
Unique paraphrases: 2
How the paraphrases change:


,,,,,,epoch
pp_l,pp_truelabel_probs,vm_score,sts_score,reward,label_flip,
Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal Mal,0.32625,0.62917,-0.08178,0.0,1,[1]
The filmmakers skillfully evoke the sense of menace that nature holds for many urban dwellers.,0.95542,0.00000,0.99993,0.2,0,"[2, 3, 4, 5]"


Best Paraphrase


,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level
2441,556,2,the filmmakers skillfully evoke the sense of menace that nature holds for many urban dwellers .,The filmmakers skillfully evoke the sense of menace that nature holds for many urban dwellers.,0.955415,0.95542,0.955415,1,1,0,0,0.99993,0.2,-0.18261,0.036522,355,821,1,2.42716,0.0379244,0,24,16,60,16,0.913291,0.152706,0.0775141,0.00728478,0.0650501,0.0750113,0.0185993,0.0293334,5.6915e-05,1.51596,0.0860494,2.04705e-05,1.73226e-06,2,1,1,15,1,1,0,79,15,1,1,0,79,0,0,0,0,0,[0],[the],[1],[The],"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]",[filmmakers skillfully evoke the sense of menace that nature holds for many urban dwellers.],1,1,1,1,False,True,False,1


## Looking at common removals and insertions 

In [ ]:
def get_common_removals_and_insertions(df_concat): 
    idx = df_concat[['data_split','orig_l', 'pp_l']].drop_duplicates().index
    df_unique_pp = df_concat[['data_split','orig_l', 'pp_l','insertions', 'removals']].iloc[idx]
    def flatten_list(l): return [item for sublist in l for item in sublist] 
    removals_flat   =  flatten_list(df_unique_pp['removals'].values)
    insertions_flat =  flatten_list(df_unique_pp['insertions'].values)
    return pd.value_counts(removals_flat), pd.value_counts(insertions_flat)

In [ ]:
df_concat = _prepare_df_concat(df_d)
removals, insertions = get_common_removals_and_insertions(df_concat)

print("\n#### REMOVALS ####\n")
print(removals.head(30))
print("\n#### INSERTIONS ####\n")
print(insertions.head(30))


#### REMOVALS ####

the       3811
a         2717
it        1943
,         1544
i         1223
.         1145
..         963
this       910
...        784
an         639
there      489
"          476
if         440
'          401
what       360
]          353
--         282
as         282
-          250
while      247
one        238
with       215
even       193
:          191
;          184
?          182
like       179
[          178
you        172
though     163
dtype: int64

#### INSERTIONS ####

The         5646
It          3109
A           3021
,           1777
I           1414
.           1197
This        1195
An           666
If           559
There        547
What         380
is           333
You          316
One          293
As           249
While        237
With         211
Even         209
and          190
Despite      179
'            173
Like         172
Not          158
For          158
Though       152
It 's        148
..           147
We           132
It is        126


Here you can look at a specific phrase and examples of where it appears. 

In [ ]:
def investigate_phrase(phrase, cname, n ): 
    mask = [phrase in strs for strs in df_concat[cname]]
    display_all(df_concat[mask].sample(n))

In [ ]:
investigate_phrase('despite', 'removals', 4)

,idx,epoch,orig_l,pp_l,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,orig_label,pp_predclass,label_flip,vm_score,sts_score,reward,pp_logp,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_loss_fn_loss_calc,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step,idx_n_unique_pp,idx_n_pp_changes,epoch_of_first_label_flip,n_words_orig,n_sentences_orig,n_punctuation_orig,n_digits_orig,n_letters_orig,n_words_pp,n_sentences_pp,n_punctuation_pp,n_digits_pp,n_letters_pp,n_words_diff,n_sentences_diff,n_punctuation_diff,n_digits_diff,n_letters_diff,removals_idx,removals,insertions_idx,insertions,unchanged_idx,unchanged,n_segments_inserted,n_segments_removed,n_tokens_inserted,n_tokens_removed,is_truncation,any_phrase_capitalised,any_phrase_decapitalised,edit_distance_token_level,data_split
41119,884,5,"despite what anyone believes about the goal of its makers , the show . . . represents a spectacular piece of theater , and there's no denying the talent of the creative forces behind it .","Despite what anyone believes about the goal of its makers, the show. represents a spectacular piece of theater, and there's no denying the talent of the creative forces behind it.",0.949238,0.95131,0.951311,1,1,0,-0.00207,0.97833,0.18341,-0.194195,0.035618,9,2330,2,5.628686,0.296247,0.000000,48,64,51,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,4,1.0,31,1,7,0,145,31,1,5,0,145,0,0,2,0,0,"[0, 16, 17]","[despite, ..]",[2],[Despite],"[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]","[what anyone believes about the goal of its makers, the show, . represents a spectacular piece of theater, and there 's no denying the talent of the creative forces behind it.]",1,2,1,3,False,True,False,3,eval_train
55757,4253,3,"despite these annoyances , the capable clayburgh and tambor really do a great job of anchoring the characters in the emotional realities of middle age .","Despite these annoyances, the capable clayburgh and tambor really do a great job of anchoring the characters in the emotional realities of middle age.",0.948049,0.94805,0.948049,1,1,0,0.00000,0.99875,0.20000,-0.052257,0.010451,40,1398,2,11.401443,0.223558,0.015625,32,64,33,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1.0,24,1,2,0,125,24,1,2,0,125,0,0,0,0,0,[0],[despite],[2],[Despite],"[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]","[these annoyances, the capable clayburgh and tambor really do a great job of anchoring the characters in the emotional realities of middle age.]",1,1,1,1,False,True,False,1,eval_train
68843,7254,4,"despite the authenticity of the trappings , the film is overblown in its plotting , hackneyed in its dialogue and anachronistic in its style .","Despite the authenticity of the trappings, the film is overblown in its plotting, hackneyed in its dialogue and anachronistic in its style.",0.933529,0.93353,0.933529,0,0,0,0.00000,0.99940,0.20000,-0.056285,0.011257,41,1864,0,4.338892,0.067795,0.000000,32,64,32,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1.0,22,1,3,0,115,22,1,3,0,115,0,0,0,0,0,[0],[despite],[2],[Despite],"[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]","[the authenticity of the trappings, the film is overblown in its plotting, hackneyed in its dialogue and anachronistic in its style.]",1,1,1,1,False,True,False,1,eval_train
72318,8053,4,"despite all the closed-door hanky-panky , the film is essentially juiceless .","Despite all the closed-door hanky-panky, the film is essentially juiceless.",0.928478,0.92848,0.928478,0,0,0,0.00000,0.99946,0.20000,-0.089336,0.017867,75,1864,0,1.849482,0.028898,0.000000,2